#Natural Language Processing Workshop<br>
##3.0 - Identifying Obesity Using Notes: Optimization Challenge!
###Names<br>10/12/2018
##Objectives
###Participants should be able to:
* Identify key words used to describe obesity
* Determine improvement in performance with *****


In [0]:
# Import libraries
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.path as path
import tensorflow as tf
import re
import seaborn as sns

# Below imports are used to print out pretty pandas dataframes
from IPython.display import display, HTML

# Imports for accessing Datathon data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery
from google.colab import files

In [0]:
auth.authenticate_user()

In [0]:
project_id='hst-953-2018'
os.environ["GOOGLE_CLOUD_PROJECT"]=project_id
# Read data from BigQuery into pandas dataframes.
def run_query(query):
  return pd.io.gbq.read_gbq(query, project_id=project_id, verbose=False, configuration={'query':{'useLegacySql': False}})



**Extra Section that should go at the beginning - just to introduce students to the different types of free text notes.**

In [14]:
df_noteevents_categories = run_query('''
SELECT CATEGORY 
FROM `physionet-data.mimiciii_notes.noteevents`
GROUP BY CATEGORY
''')

#Of note, some categories have spaces at the end, like 'Physician '
list_categories = df_noteevents_categories["CATEGORY"].tolist()

print(list_categories)
print(df_noteevents_categories)



['Physician ', 'Radiology', 'Discharge summary', 'Echo', 'ECG', 'Nursing/other', 'General', 'Rehab Services', 'Pharmacy', 'Case Management ', 'Respiratory ', 'Nutrition', 'Nursing', 'Consult', 'Social Work']
             CATEGORY
0          Physician 
1           Radiology
2   Discharge summary
3                Echo
4                 ECG
5       Nursing/other
6             General
7      Rehab Services
8            Pharmacy
9    Case Management 
10       Respiratory 
11          Nutrition
12            Nursing
13            Consult
14        Social Work


In [0]:
#This query pulls information from the derived table, height_weight.  It only uses first_height and first_weight if available.
#Patients with BMIs that are greater than or equal to 30 are categorized as obese (obese_1 = 1) which is the official definition of obese
#Obese Class 1 = 30-34.99
#Obese Class 2 = 35-39.99
#Obese Class 3 = >=40
df_noteevents_categories = run_query('''
With tt AS(
SELECT hw.icustay_id, subject_id, cast(hw.height_first as float64) as height_first, cast(hw.weight_first as float64) as weight_first
, (cast(hw.weight_first as float64))/(cast(hw.height_first as float64)/100* cast(hw.height_first as float64)/100) AS BMI
FROM `physionet-data.mimiciii_derived.heightweight` as hw
WHERE height_first is not null and weight_first is not null
)
SELECT *
, CASE WHEN (tt.bmi >= 30) THEN 1 ELSE -1 END as obese_1
FROM tt
WHERE BMI >=18.5 and BMI <= 50 and height_first >=120 AND height_first<=213 AND weight_first >40
ORDER BY tt.bmi
''')

In [45]:
df_noteevents_categories.head()

,icustay_id,subject_id,height_first,weight_first,BMI,obese_1
0,229084,1394,167.64,52.0,18.503251,-1
1,232707,10732,167.64,52.0,18.503251,-1
2,238074,24283,177.80,58.5,18.505139,-1
3,290867,26277,170.18,53.6,18.507500,-1
4,295402,21970,152.40,43.0,18.513926,-1
